# TXT Leech Bot
Run your Telegram bot in Google Colab

In [ ]:
#@title Clone Repository and Install Requirements
!git clone https://github.com/YOUR_GITHUB_USERNAME/YOUR_REPO_NAME
%cd YOUR_REPO_NAME
!pip install -r requirements.txt

In [ ]:
#@title Enter Your Credentials
API_ID = "" #@param {type:"string"}
API_HASH = "" #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}

with open('vars.py', 'w') as f:
    f.write(f'''
API_ID = "{API_ID}"
API_HASH = "{API_HASH}"
BOT_TOKEN = "{BOT_TOKEN}"
''')

In [ ]:
#@title Run Bot
!python3 main.py